# Brute Marginals

This lab is built around the process of identifying the fault with a coffee machine.

Task is to:
 1. Given the structure of a graphical model for the state of a coffee machine learn the distributions from data.
 3. Identify the most probable failure for a set of broken coffee machines.

In [1]:
%matplotlib inline

import zipfile
import io
import csv

import numpy
import matplotlib.pyplot as plt

## Coffee Machine Data Set

You can download from moodle a zip file that contains the data as a csv file. The below code will load the data directly from the zip file, so you don't have to unzip it.

Each row of the file contains a fully observed coffee machine, with the state of every random variable. The random variables are all binary, with False represented by 0 and True represented by 1. The variables are:

Failures (you're trying to detect these):  
* 0. `he` - No electricity
* 1. `fp` - Fried power supply unit
* 2. `fc` - Fried circuit board  
* 3. `wr` - Water reservoir empty
* 4. `gs` - Group head gasket forms seal  
* 5. `dp` - Dead pump  
* 6. `fh` - Fried heating element  


Mechanism (these are unobservable):  
* 7. `pw` - Power supply unit works  
* 8. `cb` - Circuit board works  
* 9. `gw` - Get water out of group head   

Diagnostic (these are the tests the mechanic can run - observable):
* 10. `ls` - Room lights switch on
* 11. `vp` - A voltage is measured across power supply unit
* 12. `lo` - Power light switches on
* 13. `wv` - Water visible in reservoir
* 14. `hp` - Can hear pump
* 15. `me` - Makes espresso
* 16. `ta` - Makes a hot, tasty espresso

For the above the number is the column number of the provided data (`dm`) and the two letter code a suggested variable name.

For if you are unfamiliar with an espresso coffee machine here is a brief description of how one works (you can ignore this if you want):
> The user puts ground coffee into a portafilter (round container with a handle and two spouts at the bottom), tamps it (compacts the coffee down), and clamps the portafilter into the group head at the front of the machine. A gasket (rubber ring) forms a seal between the portafilter and group head. A button is pressed. Water is drawn from a reservoir by a pump into a boiler. In the boiler a heating element raises the waters temperature, before the pump pushes it through the group head and into the portafilter at high pressure. The water passes through the coffee grinds and makes a tasty espresso.

The graphical model showing how the variables are related is included on moodle as `coffee machine.pdf`; here it is given as conditional probabilities:

Failures:
 * `P_he:` $P(\texttt{no electricity})$
 * `P_fp:` $P(\texttt{fried psu})$
 * `P_fc:` $P(\texttt{fried circuit board})$
 * `P_wr:` $P(\texttt{water reservoir empty})$
 * `P_gs:` $P(\texttt{group head gasket seal broken})$
 * `P_dp:` $P(\texttt{dead pump})$
 * `P_fh:` $P(\texttt{fried heating element})$

Mechanism:
 * `P_pw_he_fp:` $P(\texttt{psu works}\enspace|\enspace\texttt{no electricity},\enspace\texttt{fried psu})$
 * `P_cb_pw_fc:` $P(\texttt{circuit board works}\enspace|\enspace\texttt{psu works},\enspace\texttt{fried circuit board})$
 * `P_gw_cb_wr_dp:` $P(\texttt{get water}\enspace|\enspace\texttt{circuit board works},\enspace\texttt{water reservoir empty},\enspace\texttt{dead pump})$

Diagnostic:
 * `P_ls_he:` $P(\texttt{lights switch on}\enspace|\enspace\texttt{no electricity})$
 * `P_vp_pw:` $P(\texttt{voltage across psu}\enspace|\enspace\texttt{psu works})$
 * `P_lo_cb:` $P(\texttt{power light on}\enspace|\enspace\texttt{circuit board works})$
 * `P_wv_wr:` $P(\texttt{water visible}\enspace|\enspace\texttt{water reservoir empty})$
 * `P_hp_dp:` $P(\texttt{can hear pump}\enspace|\enspace\texttt{dead pump})$
 * `P_me_gw_gs:` $P(\texttt{makes espresso}\enspace|\enspace\texttt{get water},\enspace\texttt{group head gasket seal broken})$
 * `P_ta_me_fh:` $P(\texttt{tasty}\enspace|\enspace\texttt{makes espresso},\enspace\texttt{fried heating element})$

Note that while the model is close to what you may guess the probabilities are not absolute, to account for mistakes and unknown failures. For instance, the mechanic may make a mistake while brewing an espresso and erroneously conclude that the machine is broken when it is in fact awesome. The probabilities associated with each failure are not uniform.


In [2]:
# It may prove helpful to have a mapping between the suggested variable names and
# column indices in the provided file...
nti = dict() # 'name to index'

nti['he'] = 0
nti['fp'] = 1
nti['fc'] = 2
nti['wr'] = 3
nti['gs'] = 4
nti['dp'] = 5
nti['fh'] = 6

nti['pw'] = 7
nti['cb'] = 8
nti['gw'] = 9

nti['ls'] = 10
nti['vp'] = 11
nti['lo'] = 12
nti['wv'] = 13
nti['hp'] = 14
nti['me'] = 15
nti['ta'] = 16



# Opposite to the above - index to name...
itn = ['he', 'fp', 'fc', 'wr', 'gs', 'dp', 'fh',
       'pw', 'cb', 'gw',
       'ls', 'vp', 'lo', 'wv', 'hp', 'me', 'ta']



# For conveniance this code loads the data from the zip file,
# so you don't have to decompress it (takes a few seconds to run)...
with zipfile.ZipFile('coffee_machines.zip') as zf:
    with zf.open('coffee_machines.csv') as f:
        sf = io.TextIOWrapper(f)
        reader = csv.reader(sf)
        next(reader)
        dm = []
        for row in reader:
            dm.append([int(v) for v in row])
        dm = numpy.array(dm, dtype=numpy.int8)

print('Data: {} exemplars, {} features'.format(dm.shape[0], dm.shape[1]))   #data row and data column
print('     Broken machines  =', dm.shape[0] - dm[:,nti['ta']].sum())       #all row - row with tasty espresso
print('     Working machines =', dm[:,nti['ta']].sum())                     # row with tasty espresso
print(itn)
print(nti)
print(itn[0])
print(nti['he'])
print(nti[itn[0]])


Data: 262144 exemplars, 17 features
     Broken machines  = 122603
     Working machines = 139541
['he', 'fp', 'fc', 'wr', 'gs', 'dp', 'fh', 'pw', 'cb', 'gw', 'ls', 'vp', 'lo', 'wv', 'hp', 'me', 'ta']
{'he': 0, 'fp': 1, 'fc': 2, 'wr': 3, 'gs': 4, 'dp': 5, 'fh': 6, 'pw': 7, 'cb': 8, 'gw': 9, 'ls': 10, 'vp': 11, 'lo': 12, 'wv': 13, 'hp': 14, 'me': 15, 'ta': 16}
he
0
0


## 1. Learn Model

Below a set of variables to represent conditional probability distributions have been defined. They are a Bernoulli trial for each combination of conditional variables, given as $P(\texttt{False}|...)$ in `[0,...]` and $P(\texttt{True}|...)$ in `[1,...]` (It may be easier to think of them as boring categorical distributions).

To fit a maximum likelihood model you should first use them as counters - loop over the data set and count how many of each combination exist. You must then normalise them so that the sum over axis 0 is always 1. There is an extra mark for doing the right thing and including a prior. You may want to know that the conjugate prior to the Bernoulli trial represented as $\left[P(\texttt{False}), P(\texttt{True})\right]$ is a Beta distribution; a uniform prior would be a reasonable choice (it can be argued that the expected failure probability is low, and you should adjust the first 7 variables accordingly, but given the quantity of data available it's not going to matter).



In [3]:
# A set of variables that will ultimately represent conditional probability distributions.
# The naming convention is that P_he means P(he), or that P_ta_me_hw means P(ta|me,hw)...

P_he          = numpy.zeros(2)
P_fp          = numpy.zeros(2)
P_fc          = numpy.zeros(2)
P_wr          = numpy.zeros(2)
P_gs          = numpy.zeros(2)
P_dp          = numpy.zeros(2)
P_fh          = numpy.zeros(2)


P_pw_he_fp    = numpy.zeros((2,2,2))
P_cb_pw_fc    = numpy.zeros((2,2,2))
P_gw_cb_wr_dp = numpy.zeros((2,2,2,2))

P_ls_he       = numpy.zeros((2,2))
P_vp_pw       = numpy.zeros((2,2))
P_lo_cb       = numpy.zeros((2,2))
P_wv_wr       = numpy.zeros((2,2))
P_hp_dp       = numpy.zeros((2,2))
P_me_gw_gs    = numpy.zeros((2,2,2))
P_ta_me_fh    = numpy.zeros((2,2,2))



# It may prove conveniant to have a structure that describes the above in a computer
# readable form, including labeling what each dimension is...
ops = [(P_he, nti['he']),
       (P_fp, nti['fp']),
       (P_fc, nti['fc']),
       (P_wr, nti['wr']),
       (P_gs, nti['gs']),
       (P_dp, nti['dp']),
       (P_fh, nti['fh']),
       (P_pw_he_fp, nti['pw'], nti['he'], nti['fp']),
       (P_cb_pw_fc, nti['cb'], nti['pw'], nti['fc']),
       (P_gw_cb_wr_dp, nti['gw'], nti['cb'], nti['wr'], nti['dp']),
       (P_ls_he, nti['ls'], nti['he']),
       (P_vp_pw, nti['vp'], nti['pw']),
       (P_lo_cb, nti['lo'], nti['cb']),
       (P_wv_wr, nti['wv'], nti['wr']),
       (P_hp_dp, nti['hp'], nti['dp']),
       (P_me_gw_gs, nti['me'], nti['gw'], nti['gs']),
       (P_ta_me_fh, nti['ta'], nti['me'], nti['fh'])]



# Sensible prior are taken into account and calculated using the Beta Distribution Mean = (1+k)/(2+n)


#### 0 - 6
for i in range(0,7):
    f0 = dm.shape[0] - dm[:,nti[itn[i]]].sum()     # False counter
    t1 = dm[:,nti[itn[i]]].sum()                   # True Counter
    ops[i][0][0]=(1+f0)/(2+f0+t1)                  # Probability for condition to be false     
    ops[i][0][1]=(1+t1)/(2+f0+t1)                  # Probability for condition to be true

#### 7 - 8
for op in range(7,9):
    numer = numpy.zeros((2,2,2))
    denom = numpy.zeros((2,2))
    for n in range(dm.shape[0]):
        for i in range(0,2):
            for j in range(0,2):
                if ((dm[n, nti[itn[ops[op][2]]]]==i) and (dm[n, nti[itn[ops[op][3]]]]==j)):
                    denom[i][j] += 1               # Counter for 'n' for every condition
                for k in range(0,2):
                    if ((dm[n, nti[itn[ops[op][2]]]]==i) and (dm[n, nti[itn[ops[op][3]]]]==j) and (dm[n, nti[itn[ops[op][1]]]]==k)):
                        numer[i][j][k] += 1        # Counter for 'k' for every condition 

    for i in range(0,2):
        for j in range(0,2):
            for k in range(0,2):  

                ops[op][0][k][i][j]=(1+numer[i][j][k])/(2+denom[i][j])     # Assigning probabilities to their corresponding positions in the dictionary

                       
            

#### 9
for op in range(9,10):
    numer =numpy.zeros((2,2,2,2))
    denom =numpy.zeros((2,2,2)) 
    for n in range(dm.shape[0]):
        for i in range(0,2):
            for j in range(0,2):
                for k in range(0,2):
                    if ((dm[n, nti[itn[ops[op][2]]]]==i) and (dm[n, nti[itn[ops[op][3]]]]==j) and (dm[n, nti[itn[ops[op][4]]]]==k)):
                        denom[i][j][k] += 1           # Counter for 'n' for every condition
                    for l in range(0,2):
                        if ((dm[n, nti[itn[ops[op][2]]]]==i) and (dm[n, nti[itn[ops[op][3]]]]==j) and (dm[n, nti[itn[ops[op][4]]]]==k) and (dm[n, nti[itn[ops[op][1]]]]==l)):
                            numer[i][j][k][l] += 1    # Counter for 'k' for every condition

 

        for i in range(0,2):
            for j in range(0,2):
                for k in range(0,2):
                    for l in range(0,2):
 
                        ops[op][0][l][i][j][k] = (1+numer[i][j][k][l])/(2+denom[i][j][k])     # Assigning probabilities to their corresponding positions in the dictionary
            

            
##### 10 - 14
for op in range(10,15):
    numer = numpy.zeros((2,2))
    denom = numpy.zeros(2)
    for n in range(dm.shape[0]):
        for i in range(0,2):
            if (dm[n, nti[itn[ops[op][2]]]]==i):
                denom[i] += 1              # Counter for 'n' for every condition
            for j in range(0,2):
                if ((dm[n, nti[itn[ops[op][2]]]]==i) and (dm[n, nti[itn[ops[op][1]]]]==j)):
                    numer[i][j] += 1       # Counter for 'k' for every condition

    for i in range(0,2):
        for j in range(0,2):  
                 
                ops[op][0][j][i]=(1+numer[i][j])/(2+denom[i])      # Assigning probabilities to their corresponding positions in the dictionary
                
                
#### 15 - 16
for op in range(15,17):
    numer = numpy.zeros((2,2,2))
    denom = numpy.zeros((2,2))
    for n in range(dm.shape[0]):
        for i in range(0,2):
            for j in range(0,2):
                if ((dm[n, nti[itn[ops[op][2]]]]==i) and (dm[n, nti[itn[ops[op][3]]]]==j)):
                    denom[i][j] += 1           # Counter for 'n' for every condition
                for k in range(0,2):
                    if ((dm[n, nti[itn[ops[op][2]]]]==i) and (dm[n, nti[itn[ops[op][3]]]]==j) and (dm[n, nti[itn[ops[op][1]]]]==k)):
                        numer[i][j][k] += 1    # Counter for 'k' for every condition

    for i in range(0,2):
        for j in range(0,2):
            for k in range(0,2): 
                 
                ops[op][0][k][i][j]=(1+numer[i][j][k])/(2+denom[i][j])        # Assigning probabilities to their corresponding positions in the dictionary        

In [4]:
ops

[(array([0.99047096, 0.00952904]), 0),
 (array([0.80056152, 0.19943848]), 1),
 (array([0.98004547, 0.01995453]), 2),
 (array([0.89956742, 0.10043258]), 3),
 (array([0.95010414, 0.04989586]), 4),
 (array([0.94968071, 0.05031929]), 5),
 (array([0.97033714, 0.02966286]), 6),
 (array([[[4.81037502e-06, 9.99980683e-01],
          [9.99495714e-01, 9.98069498e-01]],
  
         [[9.99995190e-01, 1.93173257e-05],
          [5.04286435e-04, 1.93050193e-03]]]), 7, 0, 1),
 (array([[[9.99981208e-01, 9.99048525e-01],
          [4.90910787e-06, 9.99760937e-01]],
  
         [[1.87916941e-05, 9.51474786e-04],
          [9.99995091e-01, 2.39062874e-04]]]), 8, 7, 2),
 (array([[[[9.99980048e-01, 9.99611349e-01],
           [9.99817718e-01, 9.96309963e-01]],
  
          [[5.75251529e-06, 9.99892404e-01],
           [9.02397787e-01, 9.99056604e-01]]],
  
  
         [[[1.99517168e-05, 3.88651380e-04],
           [1.82282173e-04, 3.69003690e-03]],
  
          [[9.99994247e-01, 1.07596299e-04],
          

### Brute Force
The below code does exactly what you want to implement for step 2, but it brute forces it. Slow and memory inefficient of course, but lets you test it.

In [5]:
def brute_marginals(known):
    """known is a dictionary, where a random variable index existing as a key in the dictionary
    indicates it has been observed. The value obtained using the key is the value the
    random variable has been observed as. Returns a 17x2 matrix, such that [rv,0] is the
    probability of random variable rv being False, [rv, 1] the probability of being True."""
    
    # Calculate the full joint (please don't ask)...
    params = []
    for op in ops:
        params.append(op[0])
        params.append(op[1:])
    params.append(range(17))
    joint = numpy.einsum(*params)
    
    # Multiply in the known states (zero out the dimensions we know it's not)...
    for key, value in known.items():
        other = abs(value-1)
        index = [slice(None)] * len(joint.shape)
        index[key] = other
        joint[tuple(index)] = 0.0
    
    # Calculate and return all marginals...
    ret = numpy.empty((len(joint.shape), 2))
    for row in range(ret.shape[0]):
        ret[row,:] = numpy.einsum(joint, range(len(joint.shape)), [row])
    
    ret /= ret.sum(axis=1)[:,None]
    return ret

print(brute_marginals({}))
print(brute_marginals({nti['ta'] : 0}))


[[0.99047096 0.00952904]
 [0.80056152 0.19943848]
 [0.98004547 0.01995453]
 [0.89956742 0.10043258]
 [0.95010414 0.04989586]
 [0.94968071 0.05031929]
 [0.97033714 0.02966286]
 [0.20705955 0.79294045]
 [0.22287459 0.77712541]
 [0.32884651 0.67115349]
 [0.10854219 0.89145781]
 [0.21506203 0.78493797]
 [0.22367574 0.77632426]
 [0.28021332 0.71978668]
 [0.14461369 0.85538631]
 [0.42213307 0.57786693]
 [0.46711295 0.53288705]]
[[0.97961342 0.02038658]
 [0.57310261 0.42689739]
 [0.95729655 0.04270345]
 [0.7973173  0.2026827 ]
 [0.89985479 0.10014521]
 [0.89230667 0.10769333]
 [0.93651378 0.06348622]
 [0.44322143 0.55677857]
 [0.47708657 0.52291343]
 [0.70397015 0.29602985]
 [0.11830996 0.88169004]
 [0.44883505 0.55116495]
 [0.47762005 0.52237995]
 [0.36202418 0.63797582]
 [0.19628645 0.80371355]
 [0.90369034 0.09630966]
 [1.         0.        ]]


## 2. What's broken?

Simply print out what the most probable broken part of each of the below machines is. Please print it as
`dictionary key: name of broken part`, e.g. `A: wr`. This will allow automarking to be used.


In [7]:
repair = {}
repair['A'] = {nti['me'] : True}
repair['B'] = {nti['wv'] : True}
repair['C'] = {nti['wv'] : False, nti['lo'] : True}
repair['D'] = {nti['hp'] : False, nti['ta'] : False}
repair['E'] = {nti['hp'] : True, nti['wv'] : True, nti['vp']: True}



# **************************************************************** 1 mark

def find_the_broken_part(machine):
    probs = brute_marginals(repair[machine])
    failure_probs = [i[1] for i in probs]
    failure_part = itn[failure_probs.index(max(failure_probs[3:7]))]    # Broken parts probabilities are only present from index 3 to 6
    print(machine,': ', failure_part)
    
find_the_broken_part('A') 
find_the_broken_part('B') 
find_the_broken_part('C') 
find_the_broken_part('D') 
find_the_broken_part('E') 

A :  fh
B :  dp
C :  wr
D :  dp
E :  gs
